# Introduction & Business Problem

### Introduction
As a business and enterpreneurship enthusiast, i'd like to look for a business opportunities and gives my thought on:
what the people are currently need? is it highly available on the market? if it isnt, what can i do to meet those demands? how should I do it?


### Business Problem
The problem in answering above question, it would take quite sometime to gather information needed in a conventional way. That is why, we are going to need an efficient way to do it, which called data science.

Using data science, we could tackle the problems:
1. Using data to determine which business opportunity that is/are available
2. Spending money on investing required detailed and thorough analysis, which take quite some time to be done conventional way

#### Who would be interested in this project?
1. Investor looking for an opportunity to build a business
2. People that was looking for a preferred neighborhood to live


By knowing what people needs, we can use it to our advantage, to make profit from it.
To start it all, first i need to know what are the frequently visited venue category in Toronto?
Because to my understanding, the more a venue category is available, the higher demands it has received from the market.
not only that, i also need information on who visited those venues? is the population of toronto can be grouped by ethnics?
Is what these ethnics needs already widely available in Toronto?



# Data

Now that i have defined the question, what data do i need to have in order to answer said question?

a. Foursquare Location Data

b. Ethnics demographic population in Toronto

These are the data I needed, where the informations i sought were stored.
Population & Demographics data of Toronto from Wikipedia will be used to:
1. Maps the majority of ethnics that lived in Toronto
2. How many (headcount & percentage) people that belongs in that ethnics?
3. Where the majority of ethnics are located (borough & neighborhood)?

the Foursquare Location Data will be used to:
4. maps the available venue category available in Toronto
5. which venue category are amongst the top frequently visited by people of Toronto


having knowing the 1st, 2nd, and 5th data, we could determine what venue category that still has room to grow in Toronto for business opportunity, what business would potentilly thrive and where should we build it.

Meanwhile from the 3rd and 4th data, we could predict which neighborhood that potentially be the second "home" for the majority ethnics? by clustering the neighborhood based on its similarity in venue categories.